In [2]:
from bs4 import BeautifulSoup
import requests
import pymongo
import os
from splinter import Browser
import pandas as pd
import time

In [3]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [4]:
db = client.craigslist_db
collection = db.items

In [32]:
url = 'https://mars.nasa.gov/news/'

browser = Browser(driver_name="chrome")

# Retrieve page with the requests module
browser.visit(url)
response = browser.html

# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response, 'lxml')

In [36]:
# Find first news title
title = soup.find("div", {"class": "content_title"}).text
print("The first news title is: " + title)

The first news title is: NASA Brings Mars Landing to Viewers Everywhere


In [43]:
# Find first paragraph text

news_teaser = soup.find("div", {"class": "article_teaser_body"}).text
print("The paragraph text is: " + news_teaser)

The paragraph text is: NASA's InSight lander is scheduled to touch down on the Red Planet at approximately noon PST  on Nov. 26, with a new suite of instruments to probe below the Martian surface.


In [23]:
elem = soup.find("div", {"class": "article_teaser_body"})

In [24]:
elem.attrs

{'class': ['article_teaser_body']}

In [25]:
# Find image url for current featured image

url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

browser = Browser(driver_name="chrome")

browser.visit(url)
response = browser.html

soup = BeautifulSoup(response, 'lxml')

In [27]:
# featured_image_url
featured_image = soup.find("a", {"class": "button fancybox"})

In [48]:
link = featured_image.attrs["data-fancybox-href"]
print("The image url is: https://www.jpl.nasa.gov" + link)

#full_link = 

The image url is: https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16729_ip.jpg


In [49]:
# Find latest tweet 

url = 'https://twitter.com/marswxreport?lang=ens'

browser = Browser(driver_name="chrome")

browser.visit(url)
response = browser.html

soup = BeautifulSoup(response, 'lxml')


In [51]:
latest_tweet = soup.find("p", {"class": "TweetTextSize"}).text
print(latest_tweet)

Sol 2227 (2018-11-11), high -3C/26F, low -72C/-97F, pressure at 8.63 hPa, daylight 06:21-18:38


In [54]:
# Scrape the table containing facts about the planet including Diameter, Mass, etc
url = 'https://space-facts.com/mars/'

browser = Browser(driver_name="chrome")

browser.visit(url)
response = browser.html

soup = BeautifulSoup(response, 'lxml')

In [72]:
#Grab table

url = 'https://space-facts.com/mars/'

mars_table = pd.read_html(url, header=0)
mars_table[0]

,Equatorial Diameter:,"6,792 km"
0,Polar Diameter:,"6,752 km"
1,Mass:,6.42 x 10^23 kg (10.7% Earth)
2,Moons:,2 (Phobos & Deimos)
3,Orbit Distance:,"227,943,824 km (1.52 AU)"
4,Orbit Period:,687 days (1.9 years)
5,Surface Temperature:,-153 to 20 °C
6,First Record:,2nd millennium BC
7,Recorded By:,Egyptian astronomers


In [20]:
#Find high-res images

url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser = Browser(driver_name="chrome")

browser.visit(url)
response = browser.html

soup = BeautifulSoup(response, 'lxml')

In [21]:
highres_image = soup.find_all("a", {"class": "itemLink product-item"})
print(highres_image)
#len(highres_image)

[<a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/dfaf3849e74bf973b59eb50dab52b583_cerberus_enhanced.tif_thumb.png"/></a>, <a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a>, <a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/7677c0a006b83871b5a2f66985ab5857_schiaparelli_enhanced.tif_thumb.png"/></a>, <a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><h3>Schiaparelli Hemisphere Enhanced</h3></a>, <a class="itemLink product-item" href="/search/map/Mars/Viking/syrtis_major_enhanced"><img alt="Syrtis Major Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/aae41197e40d6d4f3ea557f8cfe51d15_syrtis_major_enhanced.tif_thumb.png"/></a>, <a 

In [23]:
highres_image[1].text

'Cerberus Hemisphere Enhanced'

In [26]:
#Grab hemisphere image urls and store in dictionary

hemisphere_image_urls = []

for i in range (1,8,2):
    image = highres_image[i]

    link1 = "https://astrogeology.usgs.gov" + image.attrs["href"]
    title = image.text
    browser = Browser(driver_name="chrome")

    browser.visit(link1)
    response = browser.html

    soup = BeautifulSoup(response, 'lxml')
    
    images = soup.find_all("a")

    for image in images: 
        if image.text == "Sample":
            image_url = image.attrs["href"]
            break
    hemisphere_image_urls.append({"title": title, "image_url": image_url})
    

In [27]:
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
